In [1]:
# Setup and imports
import sys
sys.path.insert(0, '/workspaces/wiki3-kg-project')

import dspy
import json
import requests
import re
from pathlib import Path
from random import shuffle, seed as random_seed
from tqdm import tqdm

from ontological_engineer import (
    configure_lm,
    StatementExtractor,
    StatementQualityJudge,
)
from ontological_engineer.judges import statement_quality_metric
from ontological_engineer.training.bootstrap import (
    load_chunks_from_notebook,
    load_facts_from_notebook,
    create_training_examples,
)

## 1. Configure Language Model

Connect to LM Studio running Qwen-30B (or your preferred model).

In [2]:
# Configure the LM (defaults to Qwen-30B via LM Studio)
lm = configure_lm(
    model="qwen/qwen3-coder-30b",
    api_base="http://host.docker.internal:1234/v1",
    temperature=0.7,
)

print(f"Configured LM: {lm}")

Configured LM: <dspy.clients.lm.LM object at 0xffff7c0dc2b0>


## 2. Load Few-Shot Examples (Albert Einstein)

Albert Einstein is our gold-standard example. These chunks and their extracted facts
serve as few-shot demonstrations for the extractor and judge.

In [3]:
# Load Albert Einstein data for few-shot examples
fewshot_dir = Path("/workspaces/wiki3-kg-project/data/albert_einstein/20251218_231446")

fewshot_chunks = load_chunks_from_notebook(fewshot_dir / "chunks.ipynb")
fewshot_facts = load_facts_from_notebook(fewshot_dir / "facts.ipynb")

print(f"Loaded {len(fewshot_chunks)} chunks from Albert Einstein")
print(f"Loaded {len(fewshot_facts)} fact sets")

# Create few-shot examples
fewshot_examples = create_training_examples(fewshot_chunks, fewshot_facts)
print(f"Created {len(fewshot_examples)} few-shot examples")

Loaded 63 chunks from Albert Einstein
Loaded 19 fact sets
Created 19 few-shot examples


In [4]:
# Show a few-shot example
if fewshot_examples:
    ex = fewshot_examples[0]
    print("Sample few-shot example:")
    print(f"  Context: {ex.section_context}")
    print(f"  Text: {ex.chunk_text[:200]}...")
    print(f"  Statements: {len(ex.statements)} items")
    for stmt in ex.statements[:3]:
        print(f"    - {stmt}")

Sample few-shot example:
  Context: Albert Einstein > Introduction
  Text: Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist best known for developing the theory of relativity. Einstein also made important contributions to quantum theory...
  Statements: 28 items
    - Albert Einstein was a German-born theoretical physicist.
    - Albert Einstein developed the theory of relativity.
    - Albert Einstein made important contributions to quantum theory.


## 3. Load Wikipedia Sample for Training

Load the 100-page Wikipedia sample. We'll fetch article content and chunk it for training.

In [5]:
# Load the Wikipedia sample
sample_file = Path("/workspaces/wiki3-kg-project/data/training/wikipedia_sample.json")

with open(sample_file) as f:
    wiki_sample = json.load(f)

print(f"Loaded {len(wiki_sample['pages'])} Wikipedia pages")
print(f"Sampling method: {wiki_sample['metadata']['sampling_method']}")
print(f"\nFirst 10 pages:")
for p in wiki_sample['pages'][:10]:
    print(f"  - {p['title']} ({p['views']:,} views)")

Loaded 100 Wikipedia pages
Sampling method: power_law

First 10 pages:
  - Zohran Mamdani (9,344,963 views)
  - ChatGPT (3,639,485 views)
  - James A. Garfield (3,524,531 views)
  - 1989 Tiananmen Square protests and massacre (2,867,005 views)
  - 2025 Bihar Legislative Assembly election (2,555,071 views)
  - Mira Nair (2,503,516 views)
  - Dick Cheney (2,186,840 views)
  - 2026 FIFA World Cup (2,155,565 views)
  - 1xBet (1,831,684 views)
  - Survivor Series: WarGames (2025) (1,590,263 views)


## 4. Fetch and Chunk Wikipedia Pages

For each page in our sample, fetch the content and break it into chunks.
We use the same chunking strategy as the original pipeline.

In [6]:
def fetch_wikipedia_content(title):
    """
    Fetch the plain text content of a Wikipedia article.
    """
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
        "format": "json",
    }
    headers = {"User-Agent": "Wiki3.ai OntologicalEngineer/0.1"}
    
    try:
        response = requests.get(url, params=params, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        pages = data.get("query", {}).get("pages", {})
        for page_id, page_data in pages.items():
            if page_id == "-1":
                return None
            return page_data.get("extract", "")
    except Exception as e:
        print(f"  Error fetching {title}: {e}")
        return None
    
    return None


def chunk_article(title, text, max_chunk_size=1500):
    """
    Split article into chunks by section.
    """
    if not text:
        return []
    
    chunks = []
    
    # Split by section headers (== Header ==)
    sections = re.split(r'\n(==+\s+.+?\s+==+)\n', text)
    
    current_section = title
    current_text = ""
    
    for i, part in enumerate(sections):
        # Check if this is a header
        if re.match(r'==+\s+.+?\s+==+', part):
            # Save previous chunk if it has content
            if current_text.strip():
                chunks.append({
                    "text": current_text.strip(),
                    "section_context": f"{title} > {current_section}",
                })
            
            # Update section name
            current_section = part.strip('= \n')
            current_text = ""
        else:
            current_text += part
            
            # If chunk gets too large, split it
            while len(current_text) > max_chunk_size:
                # Find a good break point (paragraph)
                break_point = current_text.rfind('\n\n', 0, max_chunk_size)
                if break_point == -1:
                    break_point = current_text.rfind('. ', 0, max_chunk_size)
                if break_point == -1:
                    break_point = max_chunk_size
                
                chunk_text = current_text[:break_point].strip()
                if chunk_text:
                    chunks.append({
                        "text": chunk_text,
                        "section_context": f"{title} > {current_section}",
                    })
                current_text = current_text[break_point:].strip()
    
    # Don't forget the last chunk
    if current_text.strip():
        chunks.append({
            "text": current_text.strip(),
            "section_context": f"{title} > {current_section}",
        })
    
    return chunks


# Test on one page
test_title = wiki_sample['pages'][0]['title']
test_content = fetch_wikipedia_content(test_title)
if test_content:
    test_chunks = chunk_article(test_title, test_content)
    print(f"Test: '{test_title}' -> {len(test_chunks)} chunks")
    if test_chunks:
        print(f"First chunk: {test_chunks[0]['text'][:200]}...")

Test: 'Zohran Mamdani' -> 20 chunks
First chunk: Zohran Kwame Mamdani (born October 18, 1991) is an American politician who is the mayor-elect of New York City. A member of the Democratic Party and the Democratic Socialists of America, he is set to ...


In [7]:
# Fetch and chunk all pages in sample (limit to first N for faster iteration)
MAX_PAGES = 20  # Increase for full training run
MIN_CHUNK_LENGTH = 100  # Skip very short chunks

training_chunks = []
pages_processed = 0

print(f"Fetching content for {min(MAX_PAGES, len(wiki_sample['pages']))} pages...")

for page in tqdm(wiki_sample['pages'][:MAX_PAGES]):
    content = fetch_wikipedia_content(page['title'])
    if content:
        chunks = chunk_article(page['title'], content)
        # Filter short chunks
        chunks = [c for c in chunks if len(c['text']) >= MIN_CHUNK_LENGTH]
        training_chunks.extend(chunks)
        pages_processed += 1

print(f"\nProcessed {pages_processed} pages")
print(f"Total training chunks: {len(training_chunks)}")

Fetching content for 20 pages...


100%|██████████| 20/20 [00:07<00:00,  2.74it/s]


Processed 20 pages
Total training chunks: 879


## 5. Initialize Extractor with Few-Shot Examples

Create the statement extractor and provide Albert Einstein examples as demonstrations.

In [8]:
# Select best few-shot examples (ones with good variety of statements)
NUM_FEWSHOT = 3

# Sort by statement count to get diverse examples
sorted_fewshot = sorted(fewshot_examples, key=lambda x: len(x.statements), reverse=True)
selected_fewshot = sorted_fewshot[:NUM_FEWSHOT]

print(f"Selected {len(selected_fewshot)} few-shot examples:")
for i, ex in enumerate(selected_fewshot, 1):
    print(f"  {i}. {ex.section_context[:50]}... ({len(ex.statements)} statements)")

Selected 3 few-shot examples:
  1. Albert Einstein > Introduction... (28 statements)
  2. Albert Einstein > Life and career > Personal views... (28 statements)
  3. Albert Einstein > Introduction... (27 statements)


In [9]:
# Create extractor with few-shot demonstrations
extractor = StatementExtractor()

# In DSPy, we can provide demonstrations directly
# The few-shot examples will be used by MIPROv2 for bootstrapping
print("Extractor initialized")
print(f"Few-shot examples available: {len(selected_fewshot)}")

Extractor initialized
Few-shot examples available: 3


## 6. Test Extraction on Training Sample

Run the extractor on a few training chunks to verify it works.

In [10]:
# Test on a training chunk
if training_chunks:
    test_chunk = training_chunks[0]
    
    print(f"Testing on: {test_chunk['section_context']}")
    print(f"Text: {test_chunk['text'][:300]}...")
    print("\n" + "="*60 + "\n")
    
    result = extractor(
        chunk_text=test_chunk['text'],
        section_context=test_chunk['section_context'],
    )
    
    print(f"Extracted {len(result.statements)} statements:")
    for i, stmt in enumerate(result.statements[:10], 1):
        print(f"  {i}. {stmt}")
    if len(result.statements) > 10:
        print(f"  ... and {len(result.statements) - 10} more")

Testing on: Zohran Mamdani > Zohran Mamdani
Text: Zohran Kwame Mamdani (born October 18, 1991) is an American politician who is the mayor-elect of New York City. A member of the Democratic Party and the Democratic Socialists of America, he is set to become New York's first Muslim and Asian American mayor. Mamdani has served as a member of the New Y...


Extracted 22 statements:
  1. [Zohran Kwame Mamdani](/wiki/Zohran_Kwame_Mamdani) was born on October 18, 1991.
  2. [Zohran Kwame Mamdani](/wiki/Zohran_Kwame_Mamdani) is an American politician.
  3. [Zohran Kwame Mamdani](/wiki/Zohran_Kwame_Mamdani) is the mayor-elect of [New York City](/wiki/New_York_City).
  4. [Zohran Kwame Mamdani](/wiki/Zohran_Kwame_Mamdani) is a member of the [Democratic Party](/wiki/Democratic_Party).
  5. [Zohran Kwame Mamdani](/wiki/Zohran_Kwame_Mamdani) is a member of the [Democratic Socialists of America](/wiki/Democratic_Socialists_of_America).
  6. [Zohran Kwame Mamdani](/wiki/Zohran_Kwame_Mamdani) is set t

## 7. Create Training Dataset

Convert chunks into DSPy examples. For training, we need to generate initial extractions
that can be scored and optimized.

In [11]:
# Create training examples (without labels - we'll generate and judge them)
# For DSPy optimization, we just need the inputs

random_seed(42)  # For reproducibility
shuffle(training_chunks)

# Create DSPy examples from chunks
trainset = []
for chunk in training_chunks:
    ex = dspy.Example(
        chunk_text=chunk['text'],
        section_context=chunk['section_context'],
    ).with_inputs('chunk_text', 'section_context')
    trainset.append(ex)

# Split into train/dev
split_idx = int(len(trainset) * 0.8)
devset = trainset[split_idx:]
trainset = trainset[:split_idx]

print(f"Training set: {len(trainset)} examples")
print(f"Dev set: {len(devset)} examples")

Training set: 703 examples
Dev set: 176 examples


## 8. Initialize Judge with Few-Shot Guidance

The judge scores extraction quality. We use Albert Einstein examples to calibrate.

In [12]:
# Initialize judge
judge = StatementQualityJudge()

# Test judge on a known good example (Albert Einstein few-shot)
if selected_fewshot:
    test_ex = selected_fewshot[0]
    
    evaluation = judge(
        chunk_text=test_ex.chunk_text,
        section_context=test_ex.section_context,
        statements=test_ex.statements,
    )
    
    print("Judge calibration on few-shot example:")
    print(f"  Completeness:      {evaluation.completeness:.2f}")
    print(f"  Atomicity:         {evaluation.atomicity:.2f}")
    print(f"  Accuracy:          {evaluation.accuracy:.2f}")
    print(f"  Link preservation: {evaluation.link_preservation:.2f}")
    print(f"  ---")
    print(f"  Weighted score:    {evaluation.weighted_score:.2f}")

Judge calibration on few-shot example:
  Completeness:      0.95
  Atomicity:         0.85
  Accuracy:          0.95
  Link preservation: 1.00
  ---
  Weighted score:    0.94


## 9. Baseline Evaluation

Evaluate the unoptimized extractor on the dev set.

In [13]:
# Evaluate baseline on dev set
EVAL_SIZE = min(10, len(devset))  # Limit for speed

evaluator = dspy.Evaluate(
    devset=devset[:EVAL_SIZE],
    metric=statement_quality_metric,
    num_threads=1,
    display_progress=True,
)

baseline_extractor = StatementExtractor()
baseline_result = evaluator(baseline_extractor)

baseline_score = baseline_result.score if hasattr(baseline_result, 'score') else float(baseline_result)
print(f"\nBaseline quality score: {baseline_score:.2f}")

Average Metric: 8.72 / 10 (87.2%): 100%|██████████| 10/10 [00:00<00:00, 261.66it/s]

2025/12/20 01:50:20 INFO dspy.evaluate.evaluate: Average Metric: 8.724999999999998 / 10 (87.2%)




Baseline quality score: 87.25


## 9b. MLflow Observability Setup

MLflow provides tracing, evaluation, and human feedback tools for DSPy pipelines.

### Quick Setup (One-time)

1. **Install MLflow** (already in requirements or run cell below)
2. **Start the MLflow server** in a terminal:
   ```bash
   cd /workspaces/wiki3-kg-project
   mlflow server \
     --backend-store-uri sqlite:///mlflow.sqlite \
     --default-artifact-root ./mlflow-artifacts \
     --host 0.0.0.0 \
     --port 5000
   ```
3. **Open the UI** at http://localhost:5000 (or via VS Code port forwarding)

### What MLflow Provides
- **Tracing**: See every LM call, inputs, outputs, latency
- **Evaluation**: Compare model versions side-by-side
- **Human Feedback**: Add labels/assessments directly in the UI
- **Experiment Tracking**: Track metrics across optimization runs

In [14]:
# =============================================================================
# MLflow Setup
# =============================================================================
# Prerequisites:
#   1. Install: pip install "mlflow>=3.0"
#   2. Start server in terminal (or run: ./scripts/start_mlflow.sh):
#      mlflow server --backend-store-uri sqlite:///mlflow.sqlite \
#                    --default-artifact-root ./mlflow-artifacts \
#                    --host 0.0.0.0 --port 5000
#   3. Open UI: http://localhost:5000
# =============================================================================

import mlflow

# Configure MLflow - use localhost for local server
# For Docker: use host.docker.internal if MLflow runs on host
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

try:
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment("wiki3-kg-stage1-statements")
    
    # Enable automatic DSPy tracing - captures all LM calls, modules, predictions
    mlflow.dspy.autolog()
    
    print(f"✅ MLflow configured successfully")
    print(f"   Tracking URI: {MLFLOW_TRACKING_URI}")
    print(f"   Experiment: wiki3-kg-stage1-statements")
    print(f"\n📊 Open MLflow UI: {MLFLOW_TRACKING_URI}")
    MLFLOW_ENABLED = True
except Exception as e:
    print(f"⚠️  MLflow not available: {e}")
    print(f"\n💡 To enable MLflow, start the server:")
    print(f"   mlflow server --backend-store-uri sqlite:///mlflow.sqlite --port 5000")
    MLFLOW_ENABLED = False

2025/12/20 01:50:21 INFO mlflow.tracking.fluent: Experiment with name 'wiki3-kg-stage1-statements' does not exist. Creating a new experiment.


✅ MLflow configured successfully
   Tracking URI: http://127.0.0.1:5000
   Experiment: wiki3-kg-stage1-statements

📊 Open MLflow UI: http://127.0.0.1:5000


In [15]:
# Run evaluation with MLflow tracing (if enabled)
# Each prediction creates a trace viewable in the MLflow UI

if MLFLOW_ENABLED:
    with mlflow.start_run(run_name="baseline_evaluation"):
        # Log parameters for reproducibility
        mlflow.log_param("eval_size", EVAL_SIZE)
        mlflow.log_param("model", "qwen/qwen3-coder-30b")
        mlflow.log_param("num_fewshot", NUM_FEWSHOT)
        
        # Run extractions on dev set - each one is traced
        results = []
        for i, ex in enumerate(tqdm(devset[:EVAL_SIZE], desc="Evaluating")):
            with mlflow.start_span(name=f"example_{i}") as span:
                # Run extraction
                pred = baseline_extractor(
                    chunk_text=ex.chunk_text,
                    section_context=ex.section_context,
                )
                
                # Run judge
                eval_result = judge(
                    chunk_text=ex.chunk_text,
                    section_context=ex.section_context,
                    statements=pred.statements,
                )
                
                # Log to span for MLflow UI review
                span.set_inputs({
                    "chunk_text": ex.chunk_text[:500],
                    "section_context": ex.section_context,
                })
                span.set_outputs({
                    "statements": list(pred.statements),
                    "completeness": float(eval_result.completeness),
                    "atomicity": float(eval_result.atomicity),
                    "accuracy": float(eval_result.accuracy),
                    "link_preservation": float(eval_result.link_preservation),
                    "weighted_score": float(eval_result.weighted_score),
                    "reasoning": eval_result.reasoning,
                })
                
                results.append({
                    "index": i,
                    "score": float(eval_result.weighted_score),
                })
        
        # Log aggregate metrics
        avg_score = sum(r["score"] for r in results) / len(results)
        mlflow.log_metric("avg_quality_score", avg_score)
        
        print(f"\n✅ Evaluation complete!")
        print(f"   Average score: {avg_score:.2f}")
        print(f"   Traces logged: {len(results)}")
        print(f"\n📊 Review in MLflow UI: {MLFLOW_TRACKING_URI}")
        print(f"   → Click 'Traces' tab to see all predictions")
        print(f"   → Click individual traces to review inputs/outputs")
        print(f"   → Use 'Feedback' to add human labels")
else:
    print("⏭️  Skipping MLflow evaluation (server not running)")
    print("   Run baseline evaluation with dspy.Evaluate instead")

Evaluating: 100%|██████████| 10/10 [00:02<00:00,  4.97it/s]



✅ Evaluation complete!
   Average score: 0.87
   Traces logged: 10

📊 Review in MLflow UI: http://127.0.0.1:5000
   → Click 'Traces' tab to see all predictions
   → Click individual traces to review inputs/outputs
   → Use 'Feedback' to add human labels
🏃 View run baseline_evaluation at: http://127.0.0.1:5000/#/experiments/1/runs/aa12844cba7a46b28725ae1e0207729a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


[Trace(trace_id=tr-6694c34310ba58e3d2762bdc1d34d08e), Trace(trace_id=tr-715629eee893be3d7354ea6f61607459), Trace(trace_id=tr-735435ea68949b8d00af5b3a2812859a), Trace(trace_id=tr-a5cb63a2398d1ca68b6870b51d61fac3), Trace(trace_id=tr-1497d6587010f7197e695d0d8a3c3b5e), Trace(trace_id=tr-7872bdeb2cd94cbbc19ad58cc35b1c8c), Trace(trace_id=tr-f1f8343ea99f131849c8a43f7ed70ed7), Trace(trace_id=tr-c1a6423b9f64eeed5c9d927d84b871bb), Trace(trace_id=tr-17dc8eff687213f98d60593603802b70), Trace(trace_id=tr-ebff8d1530cbd7556232b17a25074181)]

In [16]:
# Alternative: Use dspy.inspect_history() for quick debugging
# This shows recent LM calls without needing MLflow server

print("Recent LM calls (use MLflow UI for full traces):")
print("=" * 60)
dspy.inspect_history(n=2)

Recent LM calls (use MLflow UI for full traces):




[2025-12-20T01:50:23.782466]

System message:

Your input fields are:
1. `chunk_text` (str): Wikipedia article chunk with markdown links preserved
2. `section_context` (str): Breadcrumb showing location: Article > Section > Subsection
Your output fields are:
1. `reasoning` (str): 
2. `statements` (list[str]): List of atomic statements, each preserving [Entity](/wiki/...) links
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## chunk_text ## ]]
{chunk_text}

[[ ## section_context ## ]]
{section_context}

[[ ## reasoning ## ]]
{reasoning}

[[ ## statements ## ]]
{statements}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Extract atomic, verifiable statements from Wikipedia text.
        
        Each statement must:
        - Be self-c

### MLflow Evaluation with Human Feedback

Use MLflow's evaluation API to systematically review predictions and collect human labels.
The MLflow UI provides a proper interface for reviewing and annotating.

In [17]:
# Create evaluation dataset for MLflow
import pandas as pd

eval_data = []
for i, ex in enumerate(devset[:EVAL_SIZE]):
    eval_data.append({
        "index": i,
        "chunk_text": ex.chunk_text,
        "section_context": ex.section_context,
    })

eval_df = pd.DataFrame(eval_data)
print(f"Created evaluation dataset with {len(eval_df)} examples")
eval_df.head()

Created evaluation dataset with 10 examples


,index,chunk_text,section_context
0,0,In an interview with Wired magazine during the...,Charlie Kirk > Republican and pro-Trump activism
1,1,". He has described his upbringing as ""privileg...",Zohran Mamdani > Early life and education
2,2,"Wikipedia has spawned several sister projects,...",Wikipedia > Sister projects
3,3,". Throughout the demonstrations, broadcasts by...",1989 Tiananmen Square protests and massacre > ...
4,4,The visual effects for Wicked were made by Ind...,Wicked (2024 film) > Post-production and visua...


In [18]:
# Run MLflow evaluation with custom metrics
# This creates a structured evaluation run with results viewable in UI

def extract_statements(row):
    """Model function for MLflow evaluate."""
    pred = baseline_extractor(
        chunk_text=row["chunk_text"],
        section_context=row["section_context"],
    )
    return {"statements": list(pred.statements)}

def quality_metric(row, output):
    """Custom metric using our judge."""
    eval_result = judge(
        chunk_text=row["chunk_text"],
        section_context=row["section_context"],
        statements=output["statements"],
    )
    return {
        "completeness": float(eval_result.completeness),
        "atomicity": float(eval_result.atomicity),
        "accuracy": float(eval_result.accuracy),
        "link_preservation": float(eval_result.link_preservation),
        "weighted_score": float(eval_result.weighted_score),
    }

with mlflow.start_run(run_name="mlflow_evaluation"):
    eval_results = mlflow.evaluate(
        model=extract_statements,
        data=eval_df,
        extra_metrics=[
            mlflow.metrics.make_metric(
                eval_fn=quality_metric,
                name="quality",
                greater_is_better=True,
            )
        ],
    )
    
    print(f"\n✅ MLflow evaluation complete!")
    print(f"   View results in MLflow UI: {MLFLOW_TRACKING_URI}")

/workspaces/wiki3-kg-project/.venv/lib/python3.10/site-packages/mlflow/models/evaluation/deprecated.py:9: FutureWarning: The `mlflow.evaluate` API has been deprecated as of MLflow 3.0.0. Please use these new alternatives:

 - For traditional ML or deep learning models: Use `mlflow.models.evaluate`, which maintains full compatibility with the original `mlflow.evaluate` API.

 - For LLMs or GenAI applications: Use the new `mlflow.genai.evaluate` API, which offers enhanced features specifically designed for evaluating LLMs and GenAI applications.

  warnings.warn(
/workspaces/wiki3-kg-project/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic 

🏃 View run mlflow_evaluation at: http://127.0.0.1:5000/#/experiments/1/runs/059f8afc91b2416d81056c83f9c6071e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


MlflowException: Metric 'quality': Error:
KeyError('chunk_text')
Traceback (most recent call last):
  File "/workspaces/wiki3-kg-project/.venv/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'chunk_text'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/workspaces/wiki3-kg-project/.venv/lib/python3.10/site-packages/mlflow/models/evaluation/default_evaluator.py", line 709, in _test_first_row
    metric_value = metric.evaluate(eval_fn_args)
  File 

Trace(trace_id=tr-260a5962dd81b7f57d5911c6a8f1e091)

In [ ]:
# View evaluation results summary
if eval_results:
    print("Evaluation Metrics:")
    print("=" * 40)
    for metric_name, value in eval_results.metrics.items():
        print(f"  {metric_name}: {value}")
    
    print(f"\n📊 Full results table available in MLflow UI")
    print(f"   - Click on the run to see detailed traces")
    print(f"   - Use the Evaluation tab for side-by-side comparison")
    print(f"   - Add human labels directly in the UI")

### Export Annotations from MLflow

After reviewing and labeling in the MLflow UI, export your annotations for judge improvement.

In [ ]:
# Load annotations from MLflow (after you've labeled them in the UI)
# MLflow stores feedback as assessments on traces

client = mlflow.MlflowClient()

# Get the latest evaluation run
experiment = client.get_experiment_by_name("wiki3-kg-stage1-statements")
if experiment:
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["start_time DESC"],
        max_results=1,
    )
    
    if runs:
        latest_run = runs[0]
        print(f"Latest run: {latest_run.info.run_id}")
        print(f"Metrics: {latest_run.data.metrics}")
        
        # Get traces with assessments (human feedback)
        try:
            traces = client.search_traces(
                experiment_ids=[experiment.experiment_id],
                max_results=100,
            )
            print(f"Found {len(traces)} traces")
        except Exception as e:
            print(f"Trace search error: {e}")
else:
    print("No experiment found. Run evaluation first.")

In [ ]:
# Use human feedback to improve the judge
# After collecting labels in MLflow, create DSPy training examples

# For now, save the evaluation data for later use
output_dir = Path("/workspaces/wiki3-kg-project/data/training")
output_dir.mkdir(exist_ok=True)

eval_df.to_json(output_dir / "eval_dataset.json", orient="records", indent=2)
print(f"Saved evaluation dataset to {output_dir / 'eval_dataset.json'}")

print("""
📋 Next steps for human feedback:

1. Start MLflow server:
   mlflow server --backend-store-uri sqlite:///mlflow.sqlite --port 5000

2. Open MLflow UI at http://127.0.0.1:5000

3. Navigate to the experiment 'wiki3-kg-stage1-statements'

4. Click on traces to review predictions

5. Use the feedback/assessment features to label quality

6. Export labeled data for judge improvement
""")

## 10. MIPROv2 Prompt Optimization

Use DSPy's MIPROv2 optimizer to improve the extractor's prompts.
This uses the few-shot examples to bootstrap better demonstrations.

In [ ]:
from dspy.teleprompt import MIPROv2

# Configure optimizer
optimizer = MIPROv2(
    metric=statement_quality_metric,
    num_candidates=3,  # Number of prompt candidates to try
    init_temperature=0.7,
)

# Use few-shot examples for bootstrapping demonstrations
# Use training set for optimization
TRAIN_SIZE = min(20, len(trainset))  # Limit for speed

print(f"Optimizing with {TRAIN_SIZE} training examples...")
print(f"Using {len(selected_fewshot)} few-shot demos for bootstrapping...")

optimized_extractor = optimizer.compile(
    StatementExtractor(),
    trainset=trainset[:TRAIN_SIZE],
    num_batches=2,
    max_bootstrapped_demos=NUM_FEWSHOT,
    # Provide few-shot examples as initial demos
    # demos=selected_fewshot,  # Uncomment if supported
)

print("\nOptimization complete!")

In [ ]:
# Evaluate optimized extractor
optimized_result = evaluator(optimized_extractor)
optimized_score = optimized_result.score if hasattr(optimized_result, 'score') else float(optimized_result)

print(f"Baseline score:  {baseline_score:.2f}")
print(f"Optimized score: {optimized_score:.2f}")
print(f"Improvement:     {optimized_score - baseline_score:+.2f}")

## 11. Inspect Optimized Prompts

See what prompts MIPROv2 discovered.

In [ ]:
# Inspect the optimized module
print("Optimized extractor configuration:")
print("="*60)

# Try to access the optimized signature/demos
if hasattr(optimized_extractor, 'demos'):
    print(f"\nDemonstrations: {len(optimized_extractor.demos)}")
    for i, demo in enumerate(optimized_extractor.demos[:2], 1):
        print(f"  Demo {i}: {demo.section_context[:50]}...")

# Check for any instruction changes
if hasattr(optimized_extractor, 'signature'):
    print(f"\nSignature: {optimized_extractor.signature}")

## 12. Save Results

Save the optimized extractor and training data.

In [ ]:
# Save training metadata
output_dir = Path("/workspaces/wiki3-kg-project/data/training")
output_dir.mkdir(exist_ok=True)

# Save training results
results = {
    "baseline_score": baseline_score,
    "optimized_score": optimized_score,
    "train_size": TRAIN_SIZE,
    "eval_size": EVAL_SIZE,
    "num_fewshot": NUM_FEWSHOT,
    "pages_processed": pages_processed,
    "total_chunks": len(training_chunks),
}

with open(output_dir / "stage1_results.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"Saved results to {output_dir / 'stage1_results.json'}")

In [ ]:
# Save the optimized extractor state
try:
    optimized_extractor.save(output_dir / "optimized_extractor")
    print(f"Saved optimized extractor to {output_dir / 'optimized_extractor'}")
except Exception as e:
    print(f"Could not save extractor state: {e}")
    # Alternative: save as JSON
    if hasattr(optimized_extractor, 'dump_state'):
        state = optimized_extractor.dump_state()
        with open(output_dir / "optimized_extractor_state.json", "w") as f:
            json.dump(state, f, indent=2)
        print("Saved extractor state as JSON")

In [ ]:
# Save few-shot examples for reference
fewshot_data = []
for ex in selected_fewshot:
    fewshot_data.append({
        "chunk_text": ex.chunk_text,
        "section_context": ex.section_context,
        "statements": list(ex.statements),
    })

with open(output_dir / "fewshot_examples.json", "w") as f:
    json.dump(fewshot_data, f, indent=2)

print(f"Saved {len(fewshot_data)} few-shot examples")

## Summary

This notebook:
1. Loaded Albert Einstein as few-shot examples (seed/guidance)
2. Fetched and chunked Wikipedia sample pages for training
3. Established baseline extraction quality
4. Ran MIPROv2 prompt optimization
5. Saved the optimized extractor

Next steps:
- **Stage 2**: Schema matching with optimized statements
- **Stage 3**: RDF generation training
- **Arbor GRPO**: Fine-tune the full pipeline end-to-end